# Paquetes

In [1]:
import read_data as imp
import numpy as np
import pandas as pd
import pyfolio as pf
import matplotlib.pyplot as plt
import Alphas101 as Alphas

import matplotlib as plt
from math import sqrt
from yellowbrick.cluster import KElbowVisualizer
from statistics import stdev
from pylab import plot,show
from numpy import vstack,array
from numpy.random import rand
from scipy.cluster.vq import kmeans,vq
from sklearn.cluster import KMeans
from matplotlib import pyplot as plt
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA 

'''Data Prep and Model Evaluation'''
from sklearn import preprocessing as pp
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import StratifiedKFold 
from sklearn.metrics import log_loss, accuracy_score
from sklearn.metrics import precision_recall_curve, average_precision_score
from sklearn.metrics import roc_curve, auc, roc_auc_score, mean_squared_error
from keras.utils import to_categorical
from sklearn.metrics import adjusted_rand_score
import random

'''Algos'''
import tslearn
from tslearn.utils import to_time_series_dataset
from tslearn.clustering import KShape, TimeSeriesScalerMeanVariance
from tslearn.clustering import TimeSeriesKMeans
from sklearn.cluster import AgglomerativeClustering
from pyclustering.cluster.kmedoids import kmedoids
import scipy.cluster.hierarchy as sch
from scipy.cluster.hierarchy import dendrogram, linkage
from pyclustering.cluster.clarans import clarans;
from pyclustering.utils import timedcall;
from sklearn import datasets
import operator
import calendar
import itertools as it

import time

import warnings
warnings.filterwarnings("ignore")

C:\Anaconda3\lib\site-packages\pyfolio\pos.py:27: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  'Module "zipline.assets" not found; mutltipliers will not be applied' +
WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
C:\Anaconda3\lib\site-packages\theano\configdefaults.py:560: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn("DeprecationWarning: there is no c++ compiler."
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
Using TensorFlow backend.


# Funciones Auxiliares

### TSKmeans - softdtw

In [2]:
def strategy_tskmeans_softdtw (df, n =252, K =4, metric = "softdtw"):
    
    #  metric : {“euclidean”, “dtw”, “softdtw”} (default: “euclidean”)
    
    X = TimeSeriesScalerMeanVariance(mu=0., std=1.).fit_transform(df.T.values)

    
    
    km = TimeSeriesKMeans(n_clusters=4, max_iter=200, n_init=200,tol = 1e-8, \
                      metric= metric, verbose=1, random_state=2019).fit(X)

    index = list(range(len(df.columns)))
    columns = ['strategies', 'clusters', 'selection']

    results = pd.DataFrame(index=index, columns=columns)

    results['clusters'] = km.labels_
    results['strategies'] = df.columns

    sharpeclusters = []
    dfaux = df.tail(n)
    for i in range(results['clusters'].nunique()):
        l = results.loc[results['clusters'] == i].index.values.astype(int).tolist()
        dfexp = dfaux.pct_change().iloc[:, l].sum(axis=1, skipna=True)
        r = dfexp.mean()
        s = stdev(dfexp)

        sharpeclusters.append(r / s)
    sharpeclusters = np.asanyarray(sharpeclusters)

    selection = sharpeclusters.argmax()
    cond = results['clusters'] == selection

    results['selection'] = np.where(cond, 1, 0)

    return results['strategies'][results['selection'] == 1].values.tolist()

# Backtesting

In [3]:
Estrategias = pd.read_csv('Rentabilidad de estrategias base 100.csv')
Estrategias.set_index('Date', inplace = True)
Estrategias.index = pd.to_datetime(Estrategias.index)

Est_Returns = Estrategias.pct_change().fillna(0)

In [4]:
Initial_Date = pd.to_datetime('2014-01-01')

Rebalancing_dates = []
curr_month = Initial_Date.month

the_start = np.where(Initial_Date == Estrategias.index)[0].tolist()[0]

for i in range((the_start+1), len(Estrategias.index)):
    if Estrategias.index[i].month != curr_month:
        Rebalancing_dates.append(Estrategias.index[i-1])
        curr_month = Estrategias.index[i].month

## Calculating the factors over time

In [5]:
Clustering_Returns = pd.DataFrame(index = Estrategias.index[Estrategias.index >= Initial_Date],
                                  columns = ['TSKM-S'])

TSKMS = []

Memory_time_window = 126

for i in Estrategias.index[Estrategias.index >= Initial_Date]:
            
    if i in Rebalancing_dates:
        
        end = np.where(i == Estrategias.index)[0].tolist()[0]
        ini = end - Memory_time_window
        
        # update Baskets
        
        TSKMS = strategy_tskmeans_softdtw(Estrategias.iloc[ini:end,:])
       
        # Return calculus
        
        Clustering_Returns.loc[i,'TSKM-S'] = Est_Returns.loc[i,TSKMS].sum()
    else:
        
        Clustering_Returns.loc[i,'TSKM-S'] = Est_Returns.loc[i,TSKMS].sum()

Init 1
34799.743 --> 37637.751 --> 37991.303 --> 38001.877 --> 38004.834 --> 38008.534 --> 38009.251 --> 38011.896 --> 38012.095 --> 38013.708 --> 38013.928 --> 38015.273 --> 38015.569 --> 38017.420 --> 38018.382 --> 38019.368 --> 38020.576 --> 38020.972 --> 38021.695 --> 38022.044 --> 38022.275 --> 38022.541 --> 38022.826 --> 38023.175 --> 38023.894 --> 38026.266 --> 38026.112 --> 38026.595 --> 38026.596 --> 38026.765 --> 38026.830 --> 38027.075 --> 38027.264 --> 38027.521 --> 38028.253 --> 38028.694 --> 38029.191 --> 38029.206 --> 38029.331 --> 38029.334 --> 38029.436 --> 38029.482 --> 38029.953 --> 38030.098 --> 38030.104 --> 38030.191 --> 38030.210 --> 38030.585 --> 38030.815 --> 38030.970 --> 38031.018 --> 38031.019 --> 38031.081 --> 38031.094 --> 38031.236 --> 38031.324 --> 38031.330 --> 38031.481 --> 38031.592 --> 38031.799 --> 38031.825 --> 38031.824 --> 38031.864 --> 38031.867 --> 38031.960 --> 38032.066 --> 38032.183 --> 38032.121 --> 38032.202 --> 38032.184 --> 38032.204 -->

35933.126 --> 38061.266 --> 38070.642 --> 38072.244 --> 38073.843 --> 38074.195 --> 38075.037 --> 38075.191 --> 38075.883 --> 38076.040 --> 38076.754 --> 38076.568 --> 38077.301 --> 38077.475 --> 38078.045 --> 38077.993 --> 38078.407 --> 38078.364 --> 38078.647 --> 38078.604 --> 38078.793 --> 38078.752 --> 38078.932 --> 38078.913 --> 38079.042 --> 38078.922 --> 38079.059 --> 38079.024 --> 38079.091 --> 38079.037 --> 38079.102 --> 38079.032 --> 38079.086 --> 38079.103 --> 38079.066 --> 38079.136 --> 38079.198 --> 38079.167 --> 38079.207 --> 38079.177 --> 38079.225 --> 38079.368 --> 38079.340 --> 38079.457 --> 38079.432 --> 38079.552 --> 38079.550 --> 38079.663 --> 38079.869 --> 38080.109 --> 38080.047 --> 38080.075 --> 38080.040 --> 38080.080 --> 38080.048 --> 38080.077 --> 38080.124 --> 38080.094 --> 38080.132 --> 38080.105 --> 38080.137 --> 38080.112 --> 38080.140 --> 38080.117 --> 38080.140 --> 38080.116 --> 38080.133 --> 38080.175 --> 38080.148 --> 38080.184 --> 38080.159 --> 38080.

35618.721 --> 37640.487 --> 37991.431 --> 38001.688 --> 38004.705 --> 38008.416 --> 38009.156 --> 38011.825 --> 38012.031 --> 38013.666 --> 38013.886 --> 38015.248 --> 38015.542 --> 38017.405 --> 38018.364 --> 38019.360 --> 38020.565 --> 38020.968 --> 38021.688 --> 38022.042 --> 38022.271 --> 38022.540 --> 38022.823 --> 38023.175 --> 38023.892 --> 38026.266 --> 38026.111 --> 38026.594 --> 38026.607 --> 38026.768 --> 38026.824 --> 38027.079 --> 38027.258 --> 38027.523 --> 38028.254 --> 38028.696 --> 38029.193 --> 38029.207 --> 38029.333 --> 38029.336 --> 38029.438 --> 38029.484 --> 38029.955 --> 38030.100 --> 38030.106 --> 38030.192 --> 38030.212 --> 38030.586 --> 38030.817 --> 38030.972 --> 38031.019 --> 38031.021 --> 38031.083 --> 38031.095 --> 38031.238 --> 38031.326 --> 38031.331 --> 38031.483 --> 38031.594 --> 38031.801 --> 38031.827 --> 38031.826 --> 38031.865 --> 38031.869 --> 38031.962 --> 38032.067 --> 38032.185 --> 38032.123 --> 38032.204 --> 38032.185 --> 38032.205 --> 38032.

35503.671 --> 38058.601 --> 38068.795 --> 38070.762 --> 38072.721 --> 38073.255 --> 38074.322 --> 38074.582 --> 38075.431 --> 38075.641 --> 38076.470 --> 38076.305 --> 38077.124 --> 38077.300 --> 38077.938 --> 38077.877 --> 38078.346 --> 38078.289 --> 38078.616 --> 38078.557 --> 38078.781 --> 38078.725 --> 38078.932 --> 38078.898 --> 38079.051 --> 38078.917 --> 38079.073 --> 38079.027 --> 38079.110 --> 38079.046 --> 38079.126 --> 38079.041 --> 38079.120 --> 38079.096 --> 38079.091 --> 38079.126 --> 38079.213 --> 38079.157 --> 38079.210 --> 38079.180 --> 38079.229 --> 38079.372 --> 38079.344 --> 38079.461 --> 38079.436 --> 38079.556 --> 38079.556 --> 38079.668 --> 38079.875 --> 38080.115 --> 38080.055 --> 38080.082 --> 38080.054 --> 38080.094 --> 38080.055 --> 38080.085 --> 38080.124 --> 38080.094 --> 38080.132 --> 38080.105 --> 38080.137 --> 38080.113 --> 38080.140 --> 38080.117 --> 38080.140 --> 38080.116 --> 38080.133 --> 38080.176 --> 38080.148 --> 38080.184 --> 38080.160 --> 38080.

35951.605 --> 37799.506 --> 37801.237 --> 37806.586 --> 37803.488 --> 37805.822 --> 37802.336 --> 37805.482 --> 37806.715 --> 37805.573 --> 37807.094 --> 37808.985 --> 37807.351 --> 37808.558 --> 37806.357 --> 37808.636 --> 37809.352 --> 37809.434 --> 37809.355 --> 37809.873 --> 37809.354 --> 37809.888 --> 37810.432 --> 37809.830 --> 37810.112 --> 37810.011 --> 37809.660 --> 37809.939 --> 37809.691 --> 37809.796 --> 37809.532 --> 37809.714 --> 37809.853 --> 37809.779 --> 37809.825 --> 37809.815 --> 37809.958 --> 37809.843 --> 37809.957 --> 37809.855 --> 37809.981 --> 37810.040 --> 37810.079 --> 37810.132 --> 37810.179 --> 37810.223 --> 37810.285 --> 37810.336 --> 37810.613 --> 37810.778 --> 37810.776 --> 37810.747 --> 37810.779 --> 37810.759 --> 37810.786 --> 37810.751 --> 37810.850 --> 37810.767 --> 37810.861 --> 37810.771 --> 37810.850 --> 37810.777 --> 37810.854 --> 37810.779 --> 37810.849 --> 37810.775 --> 37810.837 --> 37810.831 --> 37810.848 --> 37810.836 --> 37810.854 --> 37810.

...

35488.157 --> 38057.338 --> 38068.885 --> 38071.590 --> 38073.197 --> 38073.967 --> 38074.676 --> 38075.122 --> 38075.667 --> 38076.048 --> 38076.624 --> 38076.613 --> 38077.223 --> 38077.535 --> 38078.000 --> 38078.056 --> 38078.382 --> 38078.424 --> 38078.634 --> 38078.658 --> 38078.786 --> 38078.800 --> 38078.929 --> 38078.953 --> 38079.041 --> 38078.955 --> 38079.060 --> 38079.053 --> 38079.094 --> 38079.063 --> 38079.109 --> 38079.050 --> 38079.102 --> 38079.099 --> 38079.072 --> 38079.125 --> 38079.194 --> 38079.166 --> 38079.192 --> 38079.176 --> 38079.213 --> 38079.367 --> 38079.330 --> 38079.454 --> 38079.423 --> 38079.549 --> 38079.544 --> 38079.661 --> 38079.866 --> 38080.108 --> 38080.048 --> 38080.077 --> 38080.050 --> 38080.091 --> 38080.056 --> 38080.087 --> 38080.131 --> 38080.103 --> 38080.138 --> 38080.112 --> 38080.143 --> 38080.118 --> 38080.145 --> 38080.122 --> 38080.144 --> 38080.120 --> 38080.137 --> 38080.179 --> 38080.151 --> 38080.187 --> 38080.162 --> 38080.

34417.691 --> 36209.569 --> 36232.922 --> 36686.576 --> 37219.177 --> 37225.952 --> 37231.351 --> 37234.232 --> 37242.854 --> 37244.682 --> 37252.198 --> 37252.254 --> 37253.791 --> 37253.239 --> 37255.693 --> 37256.120 --> 37259.143 --> 37258.821 --> 37260.745 --> 37260.291 --> 37261.235 --> 37260.583 --> 37262.134 --> 37263.067 --> 37264.471 --> 37264.513 --> 37265.679 --> 37265.314 --> 37265.893 --> 37265.538 --> 37266.653 --> 37266.599 --> 37267.720 --> 37268.032 --> 37269.656 --> 37269.966 --> 37270.891 --> 37270.914 --> 37271.818 --> 37271.792 --> 37272.353 --> 37272.347 --> 37273.120 --> 37273.161 --> 37273.635 --> 37273.602 --> 37274.118 --> 37274.061 --> 37274.337 --> 37274.269 --> 37274.650 --> 37274.684 --> 37274.999 --> 37275.389 --> 37275.810 --> 37275.990 --> 37276.223 --> 37276.702 --> 37276.975 --> 37276.951 --> 37277.088 --> 37277.060 --> 37277.260 --> 37277.246 --> 37277.389 --> 37277.374 --> 37277.548 --> 37277.538 --> 37277.650 --> 37277.637 --> 37277.767 --> 37277.

35270.263 --> 37640.783 --> 37991.523 --> 38002.053 --> 38004.872 --> 38008.676 --> 38009.320 --> 38012.049 --> 38012.181 --> 38013.852 --> 38014.010 --> 38015.395 --> 38015.638 --> 38017.515 --> 38018.437 --> 38019.441 --> 38020.616 --> 38021.025 --> 38021.723 --> 38022.080 --> 38022.293 --> 38022.565 --> 38022.834 --> 38023.189 --> 38023.895 --> 38026.271 --> 38026.107 --> 38026.592 --> 38026.597 --> 38026.761 --> 38026.807 --> 38027.065 --> 38027.223 --> 38027.493 --> 38028.222 --> 38028.666 --> 38029.162 --> 38029.179 --> 38029.304 --> 38029.309 --> 38029.411 --> 38029.459 --> 38029.931 --> 38030.077 --> 38030.085 --> 38030.172 --> 38030.194 --> 38030.569 --> 38030.803 --> 38030.958 --> 38031.011 --> 38031.012 --> 38031.084 --> 38031.097 --> 38031.239 --> 38031.327 --> 38031.332 --> 38031.484 --> 38031.595 --> 38031.802 --> 38031.828 --> 38031.827 --> 38031.867 --> 38031.870 --> 38031.963 --> 38032.068 --> 38032.186 --> 38032.124 --> 38032.205 --> 38032.187 --> 38032.206 --> 38032.

34918.426 --> 37707.545 --> 37729.326 --> 37737.224 --> 37740.779 --> 37744.573 --> 37746.226 --> 37748.634 --> 37749.492 --> 37751.213 --> 37751.506 --> 37752.842 --> 37752.778 --> 37753.923 --> 37753.716 --> 37754.864 --> 37754.687 --> 37756.048 --> 37755.975 --> 37757.286 --> 37757.409 --> 37758.243 --> 37758.082 --> 37758.671 --> 37758.469 --> 37758.966 --> 37758.766 --> 37759.196 --> 37759.006 --> 37759.436 --> 37759.289 --> 37759.988 --> 37759.809 --> 37760.082 --> 37759.900 --> 37760.145 --> 37759.957 --> 37760.174 --> 37760.012 --> 37760.226 --> 37760.091 --> 37760.327 --> 37760.227 --> 37760.516 --> 37760.437 --> 37760.650 --> 37760.574 --> 37760.766 --> 37760.694 --> 37760.824 --> 37760.756 --> 37760.873 --> 37760.812 --> 37760.898 --> 37760.834 --> 37760.913 --> 37760.857 --> 37760.934 --> 37760.885 --> 37760.995 --> 37760.952 --> 37760.993 --> 37760.957 --> 37760.991 --> 37760.959 --> 37760.989 --> 37760.962 --> 37760.988 --> 37760.964 --> 37760.987 --> 37760.966 --> 37760.

KeyboardInterrupt: 

In [ ]:
Clustering_Returns = 100 * (Clustering_Returns+1).cumprod()
Clustering_Returns.to_csv('Cluster Rent Base 100 TSKM-Softdtw.csv')